In [80]:
import requests
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import datetime
import os
import pytz
from pytz import timezone
from pathlib import Path
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [91]:
# Format for time
fmt = "%Y-%m-%d %H:%M:%S"

userBot = "gpmbot"
pwdBot = "9ALjKza5GmXqxMKNf"

In [3]:
# URL for chat
rootUrl = "https://darkroom.global-precious-metals.com"

In [4]:
def drkrmLogin(user=userBot,password=pwdBot, rootUrl = rootUrl):
    """
    login tryout
    """
    apiLogin = "/api/v1/login"
    data = {"user" : user, "password" : password}
    r = requests.post(rootUrl+apiLogin,data=json.dumps(
    data), headers={'Content-Type': 'application/json'})
    if r.json()['status'] == 'success':
        return r.json()['data']['userId'], r.json()['data']['authToken'] 
    else:
        print(r.json())
        raise Exception("Something went wrong")

In [5]:
def post_message(msg, channel="#general", user=userBot,password=pwdBot, rootUrl = rootUrl):
    """
    we authenticate, then post
    """
    apiMsg = "/api/v1/chat.postMessage"
    userId, authToken = drkrmLogin(user=user, password=password, rootUrl=rootUrl)
    payload = { "channel" : channel, "text" : msg}
    response = requests.post(rootUrl+apiMsg,data=json.dumps(
    payload), headers={'Content-Type': 'application/json', 'X-Auth-Token' : authToken, 'X-USer-Id' : userId})
    return response

In [6]:
DCurl = "https://www.dukascopy.com/swiss/english/home/?utm_source=freeserv"

In [81]:
def gmtToUtc(time):
    utcTime = time.astimezone(timezone("UTC"))
    return utcTime.strftime(fmt)

def currUtcTime():
    return datetime.datetime.now(timezone('UTC')).strftime(fmt)

def toUtc(time):
    tz = pytz.timezone('UTC')
    return tz.localize(time)

toUtc(datetime.datetime.strptime("2020-01-15 03:30:44", fmt))

datetime.datetime(2020, 1, 15, 3, 30, 44, tzinfo=<UTC>)

In [9]:
def getDukasTime(soup):
    """
    Returns the extracted time as given on the DukasCopy webpage from where the DukasCopy data is being extracted from.
    
    Parameters:
    
    soup (soup): The BeautifulSoup object containing the parsed HTML representation of the DukasCopy webpage.
    
    Returns:
    date_time_obj (Datetime): Datetime object representing the time on the page when the data was taken.
    """
    tdata = soup.find_all("span", {"id": "timeUpdate"})
    dtxt = tdata[0].text
    
    date_time_obj = datetime.datetime.strptime(dtxt, ' %a, %d %b %Y %H:%M:%S GMT')
    
    return date_time_obj

In [11]:
# Settings for headless chrome

options = Options()
options.headless = True
options.add_argument('--no-proxy-server')
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")

In [82]:
def getDukasCopyData(pathToChromeDriver):
    """
    Returns a DataFrame object containing the most recent Gold Data from the DukasCopy website.
    Scrapes the data from the DukasCopy website before formatting and returning the data as a pandas DataFrame.
    
    Parameters:
    pathToChromeDriver (str): Path to the chromedriver executable to be used to open web pages.
    
    Returns:
    DCDataFrame (DataFrame): A pandas DataFrame containing the most recent Gold Data from the DukasCopy website.
    dataTime (Datetime): Datetime object representing the time at which data was fetched.
    """
    browser = webdriver.Chrome(pathToChromeDriver, options=options)
    browser.get(DCurl)
    
    DChtml = browser.page_source
    soup = BeautifulSoup(DChtml, "lxml")
    
    # Finding the exact table with all the required data
    data = soup.find_all("table", {"id": "list"})
    
    # Creating and Modifying dataframe with the table 
    DCDataFrame = pd.read_html(str(data))[0]
    
    # Add a timestamp to the data
    dataTime = toUtc(getDukasTime(soup))
    DCDataFrame["timestamp"] = dataTime
    DCDataFrame = DCDataFrame.set_index("timestamp")
    
    # Filter dataframe only to Gold Data
    DCDataFrame = DCDataFrame[DCDataFrame["Live"] == "XAU/USD"]
    
    browser.quit()
    return DCDataFrame, dataTime

In [66]:
def sendPrices(currData, timestamp):
    try:
        bidPrice = currData.iloc[0]["Bid"]
        askPrice = currData.iloc[0]["Ask"]
    except:
        print("Skipped here")
    
    formattedMsg = "GOLD PRICE as of " + str(timestamp) + "\n \nBid Price: " + str(bidPrice) + "\nAsk Price: " + str(askPrice)

    post_message(formattedMsg)

In [95]:
def run(chromeDriverPath):
    while True:
        currData, timestamp = getDukasCopyData(chromeDriverPath)
        sendPrices(currData, timestamp)
        time.sleep((24 * 60 * 60) - 5)

In [ ]:
run("chromedriver")